**Finetune tester using deepseek-r1-distill on resume data using unsloth, LoRa, and Hugging Face**

In [ ]:
!pip install unsloth

In [ ]:
from unsloth import FastLanguageModel

# Define your local model directory
new_model_local = "deepseek-r1-resume"

# Load the fine-tuned model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(new_model_local)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


FileNotFoundError: deepseek-r1-resume/*.json (invalid repository id)

In [ ]:
from unsloth import FastLanguageModel

# set model to censor-free deepseek-r1-1776
# max_seq_length: set max length of the input sequence to optimize memory usage and processing speed
# dtype: dont need to check the data type
# load_in_4bit: enhance inference, reduce memory usage, quantise model to 4bit precision

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/DeepSeek-R1-Distill-Llama-8B-unsloth-bnb-4bit",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
# target_modules: specifies the list of names of spcific classes / models we want to apply LoRA adaptation

# wrap base model with LoRA fine-tuning layers
model = FastLanguageModel.get_peft_model(
    model,
    r = 64, # define size of low-rank matrices used in adaptation
    # specify which model layer LoRA should modify
    target_modules = ['q_proj', 'k_proj', 'v_proj', 'o_proj', # Multi-Head Attention
                      'gate_proj', 'up_proj', 'down_proj'], # Feedforward Network
    lora_alpha = 16, # scaling factor for LoRA layers to prevent LoRA from overpowering the base model
    lora_dropout = 0, # prevent overfitting by randomly siabling connections during training
    # dropout = 0 => fully leverage LoRA without dropping neurons (as unsloth already did it)
    bias = 'none', # whether we need additional bias terms
    use_gradient_checkpointing = 'unsloth', # gradient checkpoint to save GPU
    random_state = 3927,
    use_rslora = False,
    loftq_config = None # no quantisation for fine-tuning because it reduces precision (need to retain high precision to learn new patterns)
)

Unsloth 2025.2.15 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


Unsloth 2025.2.15 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


Unsloth applies LoRA to 32 Transformer layers
QKV layer (Query, Key, Value layers)
O layer (Output projection)
MLP layer (feedforward layers)

Data preparation

In [ ]:
# instructions - signify main query to the LLM (i.e. the question)
# response - signify the output, mention how reponse should be tailored to specific instruction

human_prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
{}

### Response:
{}"""

In [ ]:
# signal the end of a generated response in LLM
EOS_TOKEN = tokenizer.eos_token

def formatting_human_prompts_func(examples):
    instructions = examples["prompt"]
    outputs      = examples["chosen"]
    texts = []

    for instruction, output in zip(instructions, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = human_prompt.format(instruction, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

In [ ]:
from datasets import load_dataset
dataset = load_dataset("HumanLLMs/Human-Like-DPO-Dataset", split='train')
dataset = dataset.map(formatting_human_prompts_func, batched = True)

In [ ]:
print(dataset)

Dataset({
    features: ['prompt', 'chosen', 'rejected', 'text'],
    num_rows: 10884
})


In [ ]:
for i in range(5):
  print(dataset[i]['text'])

Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
Oh, I just saw the best meme - have you seen it?

### Response:
😂 Ah, no I haven't! I'm dying to know, what's the meme about? Is it a funny cat or a ridiculous situation? Spill the beans! 🤣<｜end▁of▁sentence｜>
Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
Do you have a go-to karaoke jam?

### Response:
Oh, totally! 😄 I'm a sucker for a good ol' rock ballad. Give me some Bon Jovi any day of the week! "Livin' on a Prayer" is my go-to karaoke jam. There's just something about belting out "Oh, we're halfway there!" at the top of my lungs that gets me pumped up! 🎤 What about you, do you have a favorite karaoke song? 🎶<｜end▁of▁sentence｜>
Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
**Crafty corner** Are you good at any 

In [ ]:
def tokenize_function(examples):
  return tokenizer(
      examples['text'],
      truncation=True,
      padding='max_length',
      max_length=2048,
      return_tensors = None
  )

In [ ]:
dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/6241 [00:00<?, ? examples/s]

In [ ]:
print(dataset)

Dataset({
    features: ['resume_text', 'job_description_text', 'label', 'text', 'input_ids', 'attention_mask'],
    num_rows: 6241
})


In [ ]:
for i in range(3):
  print(dataset[i])

{'resume_text': "SummaryHighly motivated Sales Associate with extensive customer service and sales experience. Outgoing sales professional with track record of driving increased sales, improving buying experience and elevating company profile with target market.\nHighlights-Soft Skills: Public Speaking, Public Relations, Team Building, Project Management, Procedure writing, Staff Supervision and Management, Ability to interface with professionals on all levels. Accomplishments, Honors, and Activities -Board of Directors Member for the Food Bank of Corpus Christi from November 2010 to April 2013. -Held Life Insurance License -Basketball Official (Referee) High School Varsity Level.\nExperienceAccountant,08/2014-05/2015Aspirus–Owen,WI,Perform daily and routine accounting functions for two main companies and five small royalty companies. Responsibilities include but are not limited to the following: Accounts Payable, Accounts Receivable, Manage and reconcile funds for multiple banks accou

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = 'text', # column name that contains the input text for training
    max_seq_length = 2048, # max length of each training example
    dataset_num_proc = 2, # use 2 CPU processes to load and process data faster
    packing = False, # make examples separately processed (whether to combine short sample into large samples)
    args = TrainingArguments(
        per_device_train_batch_size = 2, # training examples per batch (i.e. batch size) per GPU
        gradient_accumulation_steps = 4, # step size of grdient accumulation - gradients are accumulated over 4 steps before an update
        warmup_steps = 5, # first 5 steps use smaller learning rate to stablize training
        # num_train_epoches = 1,
        max_steps = 120, # number of total optimization steps
        learning_rate = 2e-4, # initial learning rate
        fp16 = not is_bfloat16_supported(), # use fp16 precision if GPU does not support bf16
        bf16 = is_bfloat16_supported(), # use bf16 if GPU supports
        logging_steps = 1, # log trainig metrics every step
        optim = 'adamw_8bit', # use adamw-8bit as optimizer to reduce memory usage
        weight_decay = 0.01, # apply L2 regularisation to prevent overfitting
        lr_scheduler_type = 'linear', # learning rate linearly decreases from initial value to 0
        seed = 3407,
        output_dir = 'outputs', # save fine-tuned model into outputs/ folder
        report_to = "none"
    )
)


Converting train dataset to ChatML (num_proc=2):   0%|          | 0/10884 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=2):   0%|          | 0/10884 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/10884 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/10884 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 10,884 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 120
 "-____-"     Number of trainable parameters = 167,772,160


Step,Training Loss
1,2.036200
2,1.937000
3,1.773700
4,1.827000
5,1.640500
6,1.638700
7,1.513200
8,1.513600
9,1.402800
10,1.352500


Step,Training Loss
1,2.036200
2,1.937000
3,1.773700
4,1.827000
5,1.640500
6,1.638700
7,1.513200
8,1.513600
9,1.402800
10,1.352500


In [ ]:
FastLanguageModel.for_inference(model)

# convert query into structure prompt then tokenize the prompt
# return_tensors='pt' to make tokenizer return a pytorch tensor then load tensor to GPU using .to('cuda') to increase speed

inputs = tokenizer(
    [
        human_prompt.format(
            "Oh, I just saw the best meme - have you seen it?"
            "", # output - leave for generation
        )
    ]
, return_tensors = 'pt', padding = True, truncation = True).to('cuda')

# generate reponse for query with max_tokens and use_cache to speed up generation
outputs = model.generate(**inputs, max_new_tokens = 1024, use_cache=True)

#
tokenizer.batch_decode(outputs)

NameError: name 'resume' is not defined

In [ ]:
resume = '''
Professional SummaryHighly motivated Sales Associate with extensive customer service and sales experience. Outgoing sales professional with track record of driving increased sales, improving buying experience and elevating company profile with target market. SkillsAzure (2 years)SQL (2 years)NET (6 years), C++ (8 years), C (6 years)HTML, Javascript (3 years)Java, ASP, JSP (3 year)Socket Programming using C and C#Assembly Language Programming on microprocessor 8085 Work History06/2012toCurrentSoftware Engineer 2Dell|Walburg,TX,Working at Microsoft, Hyderabad from June, 2012 and am part of the MACH (Microsoft Academy of College Hires) program which mandates a change of teams every 6 months.In the past 2 years, I have worked 6 months each on - Business Intelligence project for a Reporting Application for the premier customers of Microsoft.Worked extensively on Databases, handling reporting for data more than 700GB.Developer Analytics team for a project on big data mining and analysis providing a cloud solution that collects logs, exceptions and then using machine learning, algorithms to categorize and present this data to help the developers of Azure applications to figure out earlier unknown errors.Platform Creation on Azure for the entire Marketing portfolio in Microsoft such that is maximizes scalability, availability and cost-efficiency.Migration of Employee Data Management to Azure which is a global launch of employee self-service transactions and manager self-service transactions.Have done 9 weeks of internship at Microsoft, Hyderabad from May, 2011 to July, 2011.Developed a framework for connectivity to the SQL Database and deployed it successfully.Also performed Regression Testing and Performance Testing on the applications developed by the organization.Paper Presentations: Presented a paper, "Data transfer using Wi-Fi - Methods, Security and Applications" in an International Conference "Advances in Computer and Communication Technology" held in Mumbai conducted by the IETE.Projects Done: Final Year Project - Vicinity File Drop A solution to eradicate the use of pen-drives, hard disks and other storage devices in order to transfer data between devices in close proximity using Wi-Fi.Initially, an Ad-hoc network is set up and the nodes that are in vicinity (within Wi-Fi range of each other) are identified.The file is encrypted at the sender's side and sent and is decrypted at the receiver's side.The encryption/decryption is done using the AES algorithm and other security issues like Spoofing, Sybil attack are reduced in the application.The data is sent much faster and in a secure manner with an easy-to-use interface Mini - Projects Network Sniffer" Created an application using C# that gives the information of all packets being transmitted over the network that are using TCP or UDP.The captured information is decoded from raw digital form into a human-readable format that permits users of the application to easily review the exchanged information.The Network Sniffer monitors the network and it measures the amount and type of traffic on the particular network.Information is extracted from packet headers and displayed on the basis of user settings such as filters, time, etc.Development of an Air Traffic Control System" Created an interface for the air traffic controller that manages all landing, taking off and passing-by flights in the given air space using Visual Basic as the front end and created the database in Oracle for fast query retrieval.The system automatically transfers the control of the flight from the air controller to the ground controller once the flight has landed to be directed to the correct parking bay and vice versa when the flight is ready to take off.Performance of String Search Algorithms on Heterogeneous Computing (CPU/GPU) platforms" Implemented two algorithms namely, Brute Force and Boyer-Moore using normal conventional programming and tested the performance of each.Attempted to implement the same using CUDA on the GPU platform and found that the Boyer Moore execution is much faster and can be parallelized to a greater extent.Major Achievements: Gold Medalist of the university in the Information Technology branch for securing the 1st rank amongst more than 300 students.Represented the college in an international coding contest "IEEE Xtreme" and was ranked amongst the top 500 teams in the world.The online contest was held across the globe for a period of 24 hours and was organized by the IEEE professional association.Participated in the "ACM ICPC" (International Collegiate Programming Contest) Regional Contest in the Asia region, organized by the ACM professional association.Secured 1st place in a hacking contest "Crack-o-Hack" organized by the CSI Regional Student Convention.Won the 1st Prize (Gold Medal) in a Computer Science Competition in Standard 11 for the project, "Learn Trigonometry on the Mobile" in the year 2006. EducationExpected in2012totoB-Tech|Information Technology,,GPA:Information TechnologyExpected intoto|Gold Medalist of University,,GPA:GPA: 86 GPA: 9.46Expected in2008toto|Nowrosjee Wadia College of Science,,GPA:Expected in2006totoDegree Institute/School City Year Grades|Vidya Bhavan High School,,GPA: Skillsbig data, Business Intelligence, C++, encryption, Data Management, Data transfer, Java, JSP, Javascript, machine learning, microprocessor, C#, Oracle, SQL, Visual Basic
'''

job_description = '''
Company Overview DocuSign helps organizations connect and automate how they agree. Our flagship product, eSignature, is the worlds #1 way to sign electronically on practically any device, from virtually anywhere, at any time. Today, more than a million customers and a billion users in over 180 countries use DocuSign to accelerate the process of doing business and simplify peoples lives. What you'll do Our Engineering and Tech Operations team builds and operates complex solutions for global business challenges that cross cultures, legal jurisdictions, and impacts millions of people and businesses every day. We hire people with a broad set of skills and people who want to work on creating never-been-done-before solutions at scale while ensuring world-class reliability and security. Our Agreement Cloud is a revolutionary solution that changes the way people live, work, and come to agreement. DocuSign is looking for Senior Software Engineers to join the reporting team. This team is currently working on transitioning customer-facing reports to a modern highly-scalable cloud data platform. This position will demand critical thinking skills, ability to quickly understand existing systems and the ability to work in agile environments. The ability to take smart and educated risks is essential to the success of this position. If you have all that and a passion for building quality experiences to make it easier for customers to manage their users of DocuSign, then we want to talk to you This position is an individual contributor role reporting to the Senior Manager of Engineering. Responsibility Think about how to solve problems at scale and build fault tolerant systems using micro service architectureDevelop testable, high-quality, and ship-ready code with ample test coverageWork with Product Management and other developers to understand and translate engineering requirements into design requirements and provide estimatesWork as part of a cross-site development team to drive design implementation, testing and release of products Job Designation Hybrid: Employee divides their time between in-office and remote work. Access to an office location is required. (Frequency: Minimum 2 days per week; may vary by team but will be weekly in-office expectation) Positions at DocuSign are assigned a job designation of either In Office, Hybrid or Remote and are specific to the rolejob. Preferred job designations are not guaranteed when changing positions within DocuSign. DocuSign reserves the right to change a position's job designation depending on business needs and as permitted by local law. What you bring Basic 8+ years software development experience in one or more object oriented programming languages, such as C#, Java, C++3+ years of experience with large scale data systems, such as Cassandra, Elasticsearch and Presto or equivalent3+ years of experience with large scale distributed data processing frameworks such as Apache Spark2+ years of experience with cloud environments (Azure, AWS)BS degree in Computer Science or relevant field of studyMust be located in Chicago, Seattle or the Bay area or willing to relocate Preferred Knowledge and experience in high scale, highly available (99.99+) systemsExperience with Microsoft technology stack (.NET, C#, SQL Server, etc.)Experience with SaaSExperience with large scale data processing in cloud environmentsExperience with agile and test-driven development methodologiesA passion to learn new software technologiesMS in Computer Science or relevant field of study Wage Transparency Based on applicable legislation, the below details pay ranges in the following locations: California: $146,800 - $235,025 base salary Illinois and Colorado: $139,800 - $197,400 base salary Washington and New York (including NYC metro area): $139,800 - $207,325 base salary This role is also eligible for bonus, equity and benefits. Life at DocuSign Working here DocuSign is committed to building trust and making the world more agreeable for our employees, customers and the communities in which we live and work. You can count on us to listen, be honest, and try our best to do whats right, every day. At DocuSign, everything is equal. We each have a responsibility to ensure every team member has an equal opportunity to succeed, to be heard, to exchange ideas openly, to build lasting relationships, and to do the work of their life. Best of all, you will be able to feel deep pride in the work you do, because your contribution helps us make the world better than we found it. And for that, youll be loved by us, our customers, and the world in which we live. Accommodation DocuSign provides reasonable accommodations for qualified individuals with disabilities in job application procedures. If you need such an accommodation, including if you need accommodation to properly utilize our online system, you may contact us at accommodations@docusign.com. If you experience any technical difficulties or issues during the application process, or with our interview tools, please reach out to us at taops@docusign.com for assistance. Applicant and Candidate Privacy Notice States Not Eligible for Employment This position is not eligible for employment in the following states: Alaska, Hawaii, Maine, Mississippi, North Dakota, South Dakota, Vermont, West Virginia and Wyoming.
'''